In [1]:
"""Package Loading"""
"""General"""
from webscraping_dictionaries import *
from webscraping_functions import *
import time
from dateutil import relativedelta
from datetime import datetime
from selenium import webdriver
from selenium.webdriver.common.by import By  
import requests
import smtplib
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart
import ssl
import os
#import dotenv
"""Chrome Options"""
#from selenium.webdriver import Chrome 
#from selenium.webdriver.chrome.service import Service 
#from webdriver_manager.chrome import ChromeDriverManager

"""Firefox Options"""
#from selenium.webdriver import Firefox
#from selenium.webdriver.firefox.service import Service
#from webdriver_manager.firefox import GeckoDriverManager


"""Initialize the driver"""

"""Driver Setup"""

#following steps from here: https://www.zenrows.com/blog/scraping-javascript-rendered-web-pages#installing-the-requirements
#critical help using the click method from here https://www.geeksforgeeks.org/click-element-method-selenium-python/

"""Chrome Version"""
#options = webdriver.ChromeOptions()
#options.headless = True
#options.page_load_strategy = 'none' 
#chrome_path = ChromeDriverManager().install()
#chrome_service = Service(chrome_path)
#driver = Chrome(options=options, service=chrome_service) 
#driver.implicitly_wait(5)

"""Firefox Version"""  
#options = webdriver.FirefoxOptions()
#options.headless = True
#options.add_argument("--headless=new")
#options.page_load_strategy = 'none' 
#firefox_path = GeckoDriverManager().install() 
#firefox_service = Service(firefox_path)
#driver = Firefox(options=options, service=firefox_service)
#driver.implicitly_wait(5)

'Firefox Version'

"""DEBUGGING SANDBOX"""

In [ ]:
#table this for now, since we still want to check for comp plan and ordinance keywords anyway
"Charlotte County Solar Development Page"
def charlotte_county_solar_development():
    driver.get("https://www.charlottecountyva.gov/departments/planning___zoning/solar_development.php")
    time.sleep(1)
    table_rows = driver.find_elements(By.CSS_SELECTOR,"tr")
    

In [ ]:
#come back to this another day
all_dictionaries=[agendacenter_dictionary,boarddocs_dictionary,civicclerk_dictionary,civicweb_dictionary,document_center_dictionary,escribe_dictionary,granicus_dictionary,granicus_2_dictionary,laserfiche_dictionary,legistar_dictionary,meetingstable_dictionary,novusagenda_dictionary,php_table_dictionary,primegov_dictionary,county_dictionary_single_variable,county_dictionary_double_variable,city_dictionary_single_variable,city_dictionary_double_variable]

for dictionary in all_dictionaries:
    for entry in dictionary:
        print(entry)

Finding the meetings in the page:

AgendaCenter: table_rows = driver.find_elements(By.CSS_SELECTOR, "tr[class*=catAgendaRow") (get the href in a next step)

Boarddocs: all_meetings = driver.find_elements(By.CSS_SELECTOR, "a[class*='icon prevnext meeting") (boarddocs is so specific, it may be best to leave it as standalone code instead of converting it to be modular)

CivicClerk: all_meetings = driver.find_elements(By.CSS_SELECTOR,"li[class*='cpp-MuiListItem-container'") 

CivicWeb: all_meetings = driver.find_elements(By.CSS_SELECTOR,"a[class*='list-link'")

Document Center: agenda_links = driver.find_elements(By.CSS_SELECTOR,"a[class*=pdf") (get href in the next step)

eScribe: future_meetings = driver.find_elements(By.CSS_SELECTOR,"div[class*='upcoming-meeting-container'")

Granicus v2: table_rows = driver.find_elements(By.CSS_SELECTOR, 'div[class*=RowTop') (get the specific meeting links in the next step)

Granicus: table_rows=driver.find_elements(By.CSS_SELECTOR,"tr[class*=listingRow") (get the href in a following step)

* Laserfiche: meeting_folders = driver.find_elements(By.CSS_SELECTOR,"tr")

Legistar: table_rows = driver.find_elements(By.CSS_SELECTOR,'tr[id*=ctl00_')

* MeetingsTable: all_rows = driver.find_elements(By.CSS_SELECTOR,"tr")[1:]

* NovusAgenda: table_rows = driver.find_elements(By.CSS_SELECTOR,"tr")

* OnBase: table_rows = driver.find_elements(By.CSS_SELECTOR,"tr")

* PHP Table: table_rows = driver.find_elements(By.CSS_SELECTOR,'tr')

Prime Gov: table_rows = driver.find_elements(By.CSS_SELECTOR,'tr[role*=row')

* Albemarle: table_rows = driver.find_elements(By.CSS_SELECTOR, "tr")

Alleghaney: all_agenda_links = driver.find_elements(By.CSS_SELECTOR,"a[class*='attachment-link'") (finds the links right away)

Bath: pc_links = driver.find_elements(By.CSS_SELECTOR,"a[title*='Planning'") , bos_links = driver.find_elements(By.CSS_SELECTOR,"a[title*='Board of Supervisors'")

* Bland: table_rows = driver.find_elements(By.CSS_SELECTOR,"tr")

+Brunswick: pdfs = driver.find_elements(By.CSS_SELECTOR,'a[href*=".pdf"')

+Buchanan: pdf_links = driver.find_elements(By.CSS_SELECTOR,"a[href*='.pdf")

#Buena Vista City Council: links = driver.find_elements(By.CSS_SELECTOR,"a")

*Charlotte: table_rows = driver.find_elements(By.CSS_SELECTOR,"tr")

@Clarke: all_meeting_links = driver.find_elements(By.CSS_SELECTOR,"a[class*='content_link'")[2:]

Clifton Forge: list_rows = driver.find_elements(By.CSS_SELECTOR,"li[class*=pdf")

+Covington: pdfs = driver.find_elements(By.CSS_SELECTOR,"a[href*='.pdf'")

#Craig: links = driver.find_elements(By.CSS_SELECTOR,'a')

%Emporia: all_links = driver.find_elements(By.CSS_SELECTOR,"li")

!Essex PC: all_rows = driver.find_elements(By.CSS_SELECTOR,"div[class*='views-row'")

Fairfax County BOS: meetings = driver.find_elements(By.CSS_SELECTOR,"div[class*='calendar-title'")

+Fairfax County PC: agenda_links = driver.find_elements(By.CSS_SELECTOR,"a[href*='.pdf'")

Floyd: all_meeting_rows = driver.find_elements(By.CSS_SELECTOR,"p[class*='font_7'")

$Franklin (City): agendas = driver.find_elements(By.CSS_SELECTOR, "a[href*='agenda.pdf'")

Galax: agenda_links = driver.find_elements(By.CSS_SELECTOR,"a[class*='pb_button'")

Giles BOS: agenda_link = driver.find_element(By.CSS_SELECTOR,'a[class*="qbutton"')

Grayson: links = driver.find_elements(By.CSS_SELECTOR,'a[class*="brz-a"')

+Greensville: pdf_links = driver.find_elements(By.CSS_SELECTOR,"a[href*='.pdf'")

*Henrico BOS: table_rows = driver.find_elements(By.CSS_SELECTOR,'tr')[1:]

Henrico PC: current_agendas = driver.find_elements(By.CSS_SELECTOR,"a[href*='next.pdf'")

!Highland: meetings = driver.find_elements(By.CSS_SELECTOR,"div[class*='views-row'")

*King and Queen: table_rows = driver.find_elements(By.CSS_SELECTOR,"tr")

+Lee: all_agenda_links = driver.find_elements(By.CSS_SELECTOR,"a[href*='.pdf'")

@Lexington PC: list_items = driver.find_elements(By.CSS_SELECTOR,"a[class*='content_link'")

$Loudon PC: agenda_link = driver.find_element(By.CSS_SELECTOR,"a[href*='Agenda.pdf'")

*Lunenberg: all_meeting_rows = driver.find_elements(By.CSS_SELECTOR,"tr")

*Manassas Park: table_rows = driver.find_elements(By.CSS_SELECTOR,"tr")

Nelson: all_events = driver.find_elements(By.CSS_SELECTOR,'h3')[1:]

New Kent PC: all_agenda_links = driver.find_elements(By.CSS_SELECTOR,"a[href*='Archive'")[1:]

Norton: all_agendas = driver.find_elements(By.CSS_SELECTOR,"a[href*=Archive")[1:]

*Nottoway: table_rows = driver.find_elements(By.CSS_SELECTOR,"tr")

*Pittsylvania: table_rows = driver.find_elements(By.CSS_SELECTOR,"tr")[1:]

*Prince Edward BOS: table_rows = driver.find_elements(By.CSS_SELECTOR,"tr")

Prince William PC: agenda_links = driver.find_elements(By.CSS_SELECTOR,"a[href*=Agenda")

+Richmond (County): agenda_links = driver.find_elements(By.CSS_SELECTOR, "a[href*='.pdf'")

%Southampton: list_items = driver.find_elements(By.CSS_SELECTOR,"li")

*South Boston: table_rows = driver.find_elements(By.CSS_SELECTOR,'tr')

Staunton: table_rows = driver.find_elements(By.CSS_SELECTOR,'tr[class*="meeting_widget_item"')

%Sussex: list_rows = driver.find_elements(By.CSS_SELECTOR,"li")

%Tazewell: all_rows = driver.find_elements(By.CSS_SELECTOR,"li")

Virginia Beach CC: current_agenda = driver.find_element(By.CSS_SELECTOR,"a[href*=CurrentBriefAgenda")

%Virginia Beach PC: all_rows = driver.find_elements(By.CSS_SELECTOR,'li')

Westmoreland: next_meeting = driver.find_element(By.CSS_SELECTOR,"h3")

Williamsburg: agendas = driver.find_elements(By.CSS_SELECTOR,'a[class*="document-link"')

Wythe: agenda_links = driver.find_elements(By.CSS_SELECTOR,"a[href*=package")